In [1]:
!pip install xgboost

import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
import xgboost as xgb
from sklearn.metrics import accuracy_score



In [2]:
train= pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test= pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub   = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
train.head()

train.target.value_counts()


0    32542
1      584
Name: target, dtype: int64

In [3]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')

In [4]:
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,2,45.0,1,unknown,benign,0
1,ISIC_0015719,IP_3075186,1,45.0,7,unknown,benign,0
2,ISIC_0052212,IP_2842074,1,50.0,2,nevus,benign,0
3,ISIC_0068279,IP_6890425,1,45.0,1,unknown,benign,0
4,ISIC_0074268,IP_8723313,1,55.0,7,unknown,benign,0


In [5]:
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,2,70.0,3
1,ISIC_0052349,IP_7782715,2,40.0,2
2,ISIC_0058510,IP_7960270,1,55.0,6
3,ISIC_0073313,IP_6375035,1,50.0,6
4,ISIC_0073502,IP_0589375,1,45.0,2


In [6]:
x_train = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train = train['target']


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]
# y_train = test['target']


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)

In [7]:
param = {
    'booster':'gbtree', 
    'eta': 0.3,
    'num_class': 2,
    'max_depth': 
}

epochs = 100

SyntaxError: invalid syntax (<ipython-input-7-8a7ad01bf86b>, line 6)

In [8]:
# model = xgb.train(param, 
#                   train_DMatrix, 
#                   num_boost_round=epochs)

clf = xgb.XGBClassifier(n_estimators=2000, 
                        max_depth=8, 
                        objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.15, 
                        num_class = 2, 
                        scale_pos_weight = (32542/584))



In [9]:
clf.fit(x_train, y_train)

[05:07:40] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.15, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=-1, nthread=-1, num_class=2,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=55.72260273972603,
              seed=0, subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [10]:
# predictions = model.predict(test_DMatrix)
# proba = model.predict_proba(test_DMatrix) 
clf.predict_proba(x_test)[:,1]
# clf.predict(x_test)
sub.target = clf.predict_proba(x_test)[:,1]
sub_tabular = sub.copy()

In [11]:
sub_public_merge = pd.read_csv('/kaggle/input/submission-9/submission_935.csv')
sub_mean = pd.read_csv('/kaggle/input/siim-isic-multiple-model-training-stacking-923/submission_mean.csv')

In [12]:
sub.target = sub_mean.target *0.1 + sub_public_merge.target *0.7 + sub_tabular.target *0.2

In [13]:
sub.head()
sub.to_csv('submission.csv', index = False)

In [14]:
# train_df.diagnosis.value_counts()

https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn